<a href="https://colab.research.google.com/github/skymined/study_data_analytics/blob/main/quests/EDAs/preprocessing_titanic_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [127]:
import pandas as pd
import numpy as np
df_titanic = pd.read_csv('TitanicFromDisaster_train.csv')

In [43]:
df_titanic[:2]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


## Quest 결측치 및 이상치 처리

In [44]:
df_titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1. 'Age' 열에서 결측치를 평균 나이로 채우기.

In [45]:
mean_age = df_titanic['Age'].mean()
df_titanic_1 = df_titanic

In [46]:
df_titanic_1['Age'].fillna(mean_age, inplace=True)

In [47]:
df_titanic_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          891 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### 2. 'Cabin' 열에 결측치가 있는 행을 찾고, 해당 행들의 평균 'Fare' 계산하기.

In [48]:
df_titanic_2 = df_titanic
df_titanic_2['Cabin_결측여부'] = df_titanic['Cabin'].isnull()

In [49]:
df_titanic_2_2 = df_titanic_2[['Cabin_결측여부', 'Fare']].query('Cabin_결측여부 == True')
df_titanic_2_2['Fare'].mean()

19.157325327510915

### 3. 'Embarked' 열에서 결측치가 있는 행을 찾고, 가장 많이 등장하는 항구로 채우기.

In [50]:
df_titanic['Embarked'].info()

<class 'pandas.core.series.Series'>
RangeIndex: 891 entries, 0 to 890
Series name: Embarked
Non-Null Count  Dtype 
--------------  ----- 
889 non-null    object
dtypes: object(1)
memory usage: 7.1+ KB


In [51]:
df_titanic['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [52]:
df_titanic['Embarked'].fillna('S')

0      S
1      C
2      S
3      S
4      S
      ..
886    S
887    S
888    S
889    C
890    Q
Name: Embarked, Length: 891, dtype: object

### 4. 'Fare' 열에서 평균보다 3표준편차 이상 높은 이상치를 가진 행들을 찾기.

In [62]:
Q1 = df_titanic['Fare'].describe().loc['25%']
Q3 = df_titanic['Fare'].describe().loc['75%']
max = Q3 + (Q3-Q1)
max

54.089600000000004

In [63]:
df_titanic.query(f'Fare >= {max}').sort_values(by='Fare', ascending=True)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_결측여부
356,357,1,1,"Bowerman, Miss. Elsie Edith",female,22.000000,0,1,113505,55.0000,E33,S,False
166,167,1,1,"Chibnall, Mrs. (Edith Martha Bowerman)",female,29.699118,0,1,113505,55.0000,E33,S,False
370,371,1,1,"Harder, Mr. George Achilles",male,25.000000,1,0,11765,55.4417,E50,C,False
434,435,0,1,"Silvey, Mr. William Baird",male,50.000000,1,0,13507,55.9000,E44,S,False
577,578,1,1,"Silvey, Mrs. William Baird (Alice Munger)",female,39.000000,1,0,13507,55.9000,E44,S,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
438,439,0,1,"Fortune, Mr. Mark",male,64.000000,1,4,19950,263.0000,C23 C25 C27,S,False
88,89,1,1,"Fortune, Miss. Mabel Helen",female,23.000000,3,2,19950,263.0000,C23 C25 C27,S,False
737,738,1,1,"Lesurer, Mr. Gustave J",male,35.000000,0,0,PC 17755,512.3292,B101,C,False
679,680,1,1,"Cardeza, Mr. Thomas Drake Martinez",male,36.000000,0,1,PC 17755,512.3292,B51 B53 B55,C,False


### 5. 'SibSp' (형제 자매/배우자 수)가 5 이상인 이상치를 가진 행들을 찾기.

In [64]:
df_titanic.query('SibSp >= 5').sort_values(by='SibSp', ascending=True)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Cabin_결측여부
59,60,0,3,"Goodwin, Master. William Frederick",male,11.000000,5,2,CA 2144,46.90,NaN,S,True
71,72,0,3,"Goodwin, Miss. Lillian Amy",female,16.000000,5,2,CA 2144,46.90,NaN,S,True
386,387,0,3,"Goodwin, Master. Sidney Leonard",male,1.000000,5,2,CA 2144,46.90,NaN,S,True
480,481,0,3,"Goodwin, Master. Harold Victor",male,9.000000,5,2,CA 2144,46.90,NaN,S,True
683,684,0,3,"Goodwin, Mr. Charles Edward",male,14.000000,5,2,CA 2144,46.90,NaN,S,True
159,160,0,3,"Sage, Master. Thomas Henry",male,29.699118,8,2,CA. 2343,69.55,NaN,S,True
180,181,0,3,"Sage, Miss. Constance Gladys",female,29.699118,8,2,CA. 2343,69.55,NaN,S,True
201,202,0,3,"Sage, Mr. Frederick",male,29.699118,8,2,CA. 2343,69.55,NaN,S,True
324,325,0,3,"Sage, Mr. George John Jr",male,29.699118,8,2,CA. 2343,69.55,NaN,S,True
792,793,0,3,"Sage, Miss. Stella Anna",female,29.699118,8,2,CA. 2343,69.55,NaN,S,True


### 6. Parch' (부모/자녀 수) 열에서 0이 아닌 값을 가진 행들의 평균 'Age' 계산하기.

In [73]:
df_titanic_6 = df_titanic.query('Parch != 0')
df_titanic_6['Age'].mean()

23.63451808892571

### 7. 'Fare'가 0인 행들을 찾고, 이를 평균 'Fare'로 채우기.

In [79]:
mean_7 = df_titanic['Fare'].mean()

In [86]:
df_titanic['Fare'] = df_titanic['Fare'].apply(lambda x : mean_7 if x == 0 else x)
df_titanic['Fare'].value_counts()

8.0500     43
13.0000    42
7.8958     38
7.7500     34
26.0000    31
           ..
35.0000     1
28.5000     1
6.2375      1
14.0000     1
10.5167     1
Name: Fare, Length: 248, dtype: int64

### 8. 'Pclass' (티켓 등급)가 1등급인 승객들 중 'Cabin' 열의 결측치 비율 계산하기.

In [88]:
df_titanic_8 = df_titanic.query('Pclass == 1')

In [90]:
df_titanic_8['Cabin'].isnull().sum()

40

In [93]:
df_titanic_8['Cabin'].info()

<class 'pandas.core.series.Series'>
Int64Index: 216 entries, 1 to 889
Series name: Cabin
Non-Null Count  Dtype 
--------------  ----- 
176 non-null    object
dtypes: object(1)
memory usage: 3.4+ KB


결측치 비율 = 40/216

### 9. 'Age'에는 결측치가 있고 'Fare' 은 0인 행들을 찾기.

In [110]:
df_titanic[['age_null', 'fare_null']] = df_titanic[['Age', 'Fare']].isnull()
condition_9_1 = 'age_null == True'
condition_9_2 = 'fare_null == 0 '

In [111]:
df_titanic.query(f'{condition_9_1} and {condition_9_2}')

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,age_null,fare_null
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,True,False
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,True,False
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,True,False
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,True,False
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C,True,False
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S,True,False
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S,True,False
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S,True,False


### 10. 'Sex' 열에서 값이 'male' 또는 'female'이 아닌 이상치를 가진 행들을 찾기.

In [113]:
df_titanic['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

없음.

### 11. 'Fare' 열의 결측치(값이 0)를 해당 'Pclass'의 평균 요금으로 채우기.

In [132]:
df_titanic['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [129]:
mean_P_1 = df_titanic.query('Pclass == 1')['Fare'].mean()
mean_P_2 = df_titanic.query('Pclass == 2')['Fare'].mean()
mean_P_3 = df_titanic.query('Pclass == 3')['Fare'].mean()
mean_P_1, mean_P_2, mean_P_3

(84.1546875, 20.662183152173913, 13.675550101832993)

In [130]:
def pclass_fare(df) :
  if df[1] == 1:
    df[0] = mean_P_1
  elif df[1] == 2:
    df[0] = mean_P_2
  elif df[1] == 3 :
    df[0] = mean_P_3
df_titanic['Fare'] = df_titanic[['Fare', 'Pclass']].apply(pclass_fare)

In [131]:
df_titanic['Fare']

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
      ... 
886    NaN
887    NaN
888    NaN
889    NaN
890    NaN
Name: Fare, Length: 891, dtype: object